#Introdução à inteligência artificial

##Projeto final - Classificação imagens de raio-x para identificar pneumonia

Este arquivo deve ser entregue até dia 05/07/20. Para isso, adicione o seu RA no título do arquivo e no arquivo do modelo para identificação.
A finalidade deste projeto é modificar o **classificador** de uma rede neural previamente treinada, onde fica a seu critério escolher qual arquitetura do modelo VGG você irá utilizar. 
Será avaliado quais foram as camadas utilizadas, funções de ativação, métodos para evitar *overfitting*, critério de perda, otimizador e quaisquer outros métodos que alterem os hiperparâmetros da rede.

A precisão percentual será calculada pelo algoritmo no final deste arquivo.

###Configuração inicial
Para que possamos baixar o dataset do Kaggle, crie uma conta ou associe sua conta Google. Entre em "My account" clicando no canto superior direito (que seria seu usuário), dentro da sua conta vá até a sessão "API", clique no botão "Create New API Token".

Será feito o download de um arquivo chamado "kaggle.json" que permitirá acesso aos datasets no Kaggle.
Execute a célula a seguir, ela pedirá para que você insira um arquivo, portanto selecione o "kaggle.json" que será usado para fazer o download do dataset.

In [ ]:
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!rm ~/.kaggle/*
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Execute a célula a seguir para extrair os dados:

In [ ]:
!unzip /content/chest-xray-pneumonia.zip

In [ ]:
# Importação das bibliotecas
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset

### Definição das transformações e tamanho de lote

In [ ]:

# Define as transformações, fique a vontade para adicionar mais transformações!!!
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = '/content/chest_xray'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}
data_loader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val','test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Função auxiliar
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
      plt.title(title)
    plt.pause(0.001)


# Pega um lote de treinamento
inputs, classes = next(iter(data_loader['train']))

# Faz um "grid" com as imagens
out = torchvision.utils.make_grid(inputs)

# Mostra um lote de imagens
imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Época {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Cada época tem uma fase de treino e validação
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Modelo em treinamento
            else:
                model.eval()   # Modelo em avaliação

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data_loader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zera o gradiente do otimizador
                optimizer.zero_grad()

                # Analisa somente as perdas se for no treinamento
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 'loss.backward()' + 'optimizer.step()' somente no treinamento
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Estatisticas
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Perda: {:.4f} Precisão: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Copia o modelo
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Treinamento completo em {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Melhor precisão: {:4f}'.format(best_acc))
    torch.save(best_model_wts,"model.pt")

    # Carrega os pesos do melhor modelo
    model.load_state_dict(best_model_wts)
    return model

### Modelo

Defina seu modelo utilizando o modelo [VGG](https://pytorch.org/docs/stable/torchvision/models.html) **pré-treinado**  e altere o classificador da rede neural.

In [ ]:
# TODO: Defina aqui qual VGG você utilizará, lembre-se que é aconselhavel usar a rede pré-treinada
model = NotImplemented

for param in model.parameters():
    param.requires_grad = False
   
# Quantidade de entradas do classificador antigo
num_ftrs = model.classifier[0].in_features

# TODO: Defina o novo classificador com entrada de num_ftrs e saída de len(class_names)
model.classifier = nn.Sequential(nn.Linear(num_ftrs,x),
                                 .
                                 .
                                 .

                                 nn.Linear(y,len(class_names)))

# Move o modelo para o dispositivo disponivel
model = model.to(device)

# TODO: Defina o critério
criterion = NotImplemented

# TODO: Defina o otimizador
optimizer = NotImplemented

# TODO: Defina o "scheduler"
exp_lr_scheduler = NotImplemented

In [ ]:
len(class_names)

### Treinamento

In [ ]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                    num_epochs=50)

### Carregar o modelo

In [ ]:
# Carrega o modelo 
file_name = "model.pt"
model.load_state_dict(torch.load(file_name))

### Avaliação do projeto

In [ ]:
# Faça o teste final para avaliação da rede
test_loss = 0.0
class_correct = list(0. for i in range(len(class_names)))
class_total = list(0. for i in range(len(class_names)))
batch_size = 4
train_on_gpu = torch.cuda.is_available()
model.eval()

for batch_idx, (data, target) in enumerate(data_loader['test']):
    # Move os tensores para a GPU se disponivel
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # Inferência 
    output = model(data)
    # Calcula a perda
    loss = criterion(output, target)
    # Atualiza a perda
    test_loss += loss.item()*data.size(0)
    # Convert as probabilidades para classe e escolhe somente a maior
    _, pred = torch.max(output, 1)    
    # Compara as predições com a classe verdadeira
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # Calcula a precisão para cada objeto
    for i in range(batch_size):
        label = target.data[i]
        if(correct.size==batch_size):
          class_correct[label] += correct[i].item()
          class_total[label] += 1

# Perda média
test_loss = test_loss/len(data_loader['test'].dataset)
print('Perda de teste: {:.6f}\n'.format(test_loss))

for i in range(len(class_names)):
    if class_total[i] > 0:
      name = class_names[i].capitalize()
      print('Precisão de teste de %5s: %2d%% (%2d/%2d)' % (
            name, 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Precisão de teste de %5s: N/A (sem dados de treinamento)' % (class_names[i]))
overall_acc = 100. * np.sum(class_correct) / np.sum(class_total)
print('\nPrecisão de teste geral: %6.2f%% (%3d/%3d)' % (
    overall_acc,
    np.sum(class_correct), np.sum(class_total)))

In [ ]:
 !wget -O test.jpg https://www.msdmanuals.com/-/media/manual/professional/images/m2400651_pneumonia_of_the_right_lower_lobe_science_photo_library_high_pt.jpg?la=pt&thn=0

In [ ]:
import cv2
from PIL import Image
img = Image.fromarray(cv2.imread("/content/test.jpg"))

In [ ]:
plt.imshow(Image.fromarray(img),cmap="gray")

In [ ]:
image_transform = data_transforms['test'](img)

In [ ]:
model.eval()
out = model(image_transform)

In [ ]:
print(out)